In [ ]:
!nvidia-smi

Thu Aug  7 18:45:40 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   42C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [27]:
import torch
import os
import shutil
import random
from IPython.display import Image

!pip install tqdm --upgrade
from tqdm.notebook import tqdm

# Check PyTorch GPU availability
if torch.cuda.is_available():
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"GPU Is Available: {torch.cuda.is_available()}")
else:
    print("GPU is not available.")

GPU Name: Tesla T4
GPU Is Available: True


In [28]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [29]:
!pip install ultralytics --upgrade


In [30]:
%cd /content/drive/MyDrive/Final_Defence

/content/drive/MyDrive/Final_Defence


In [31]:
!git clone https://github.com/WongKinYiu/yolov9.git

fatal: destination path 'yolov9' already exists and is not an empty directory.


In [32]:
%cd yolov9

/content/drive/MyDrive/Final_Defence/yolov9


In [33]:
!pip install -r requirements.txt

In [ ]:
trainImagePath = '/content/drive/MyDrive/Final_Defence/data/images/train/'
trainLabelPath = '/content/drive/MyDrive/Final_Defence/data/labels/train/'
valImagePath = '/content/drive/MyDrive/Final_Defence/data/images/val/'
valLabelPath = '/content/drive/MyDrive/Final_Defence/data/labels/val/'
testPath = '/content/drive/MyDrive/Final_Defence/data/test/'

In [ ]:
def train_test_split(path,neg_path=None, split=0.2):
  print("-------- PROCESS STARTED ---------")

  files = list(set([name[:-4] for name in os.listdir(path)]))   ## Removing duplicate names data...

  print(f">>> This folder has a total number of {len(files)} images <<<")
  random.seed(42)
  random.shuffle(files)

  test_size = int(len(files) * split)
  train_size = len(files) - test_size

  ## Createing requird directories

  os.makedirs(trainImagePath, exist_ok= True)
  os.makedirs(trainLabelPath, exist_ok= True)
  os.makedirs(valImagePath, exist_ok= True)
  os.makedirs(valLabelPath, exist_ok= True)

  ## Copying images to train folder

  for filex in tqdm(files[:train_size]):
    if filex == 'classes':
      continue
    shutil.copy2(path + filex + '.jpg', f"{trainImagePath}/" + filex + '.jpg')
    shutil.copy2(path + filex + '.txt', f"{trainLabelPath}/" + filex + '.txt')

  print(f">>>> Training data created with 80% split {len(files[:train_size])} images <<<<")

  if neg_path:
    neg_images = list(set([name[:4] for name in os.listdir(neg_path)]))   ## Removing duplicate names data

    for filex in tqdm(neg_images):
      shutil.copy2(neg_path + filex + '.jpg', f"{trainImagePath}/" + filex + '.jpg')

    print(f">>>> Total {len(neg_images)} negative images added to the training data <<<<")
    print(f">>>> Total training data created with {len(files[:train_size]) + len(neg_images)} images <<<<")


  ## Copying images to validation folder

  for filex in tqdm(files[train_size:]):
    if filex == 'classes':
       continue

    shutil.copy2(path + filex + '.jpg', f"{valImagePath}/" + filex + '.jpg')
    shutil.copy2(path + filex + '.txt', f"{valLabelPath}/" + filex + '.txt')

  print(f">>>> Testing data created with 80% split {len(files[train_size:])} images <<<<")

  print(">>>> TASK COMPLETE <<<<")


### for label_tag
train_test_split('/content/drive/MyDrive/yolo/yolov8/raw_data/')    ## Without negative images

In [ ]:
%cd /content/drive/MyDrive/yolo/yolov8/data/

!touch dataset.yaml
!ls /content/drive/MyDrive/yolo/yolov8/data/

In [ ]:
!wget -P /content/drive/MyDrive/Final_Defence/yolov9 https://github.com/WongKinYiu/yolov9/releases/download/v0.1/yolov9-c.pt

!wget -P /content/drive/MyDrive/Final_Defence/yolov9 https://github.com/WongKinYiu/yolov9/releases/download/v0.1/yolov9-e.pt

--2025-08-07 18:48:11--  https://github.com/WongKinYiu/yolov9/releases/download/v0.1/yolov9-c.pt
Resolving github.com (github.com)... 140.82.116.3
Connecting to github.com (github.com)|140.82.116.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/759338070/c8ca43f2-0d2d-4aa3-a074-426505bfbfb1?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-08-07T19%3A42%3A28Z&rscd=attachment%3B+filename%3Dyolov9-c.pt&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-08-07T18%3A42%3A02Z&ske=2025-08-07T19%3A42%3A28Z&sks=b&skv=2018-11-09&sig=lVXmYB96U2wBqUl1fwusV7LRtVRL4Q%2F%2B0TqYFizivj8%3D&jwt=eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc1NDU5Mjc5MSwibmJmIjoxNzU0NTkyNDkxLCJwYXRoIjoicmVsZWFzZWFzc2V0cHJvZHVjdGlvbi5ibG9iLmNvcm

In [ ]:
%cd /content/drive/MyDrive/Final_Defence/yolov9/

/content/drive/MyDrive/Final_Defence/yolov9


## Training yolov9 on custom dataset

#### Download Dataset: https://universe.roboflow.com/roboflow-100/furniture-ngpea

Paste the dataset folder in your drive

#####  1- In yolov9 repo paste this data.yaml file

##### 2- Create a copy of models/detect/yolov9.yaml file and name it yolov9_custom.yaml.
Open it and change the value of nc as per your custom model

In [ ]:
#%cd yolov9

!python train_dual.py --workers 8 --batch 8  --img 640 --epochs 100 --data /content/drive/MyDrive/Final_Defence/data/dataset.yaml --weights /content/drive/MyDrive/Final_Defence/yolov9/yolov9-e.pt --device 0 --cfg /content/drive/MyDrive/Final_Defence/yolov9/models/detect/yolov9_custom.yaml --hyp /content/drive/MyDrive/Final_Defence/yolov9/data/hyps/hyp.scratch-high.yaml

2025-08-07 19:27:17.684782: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754594837.704553   36934 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754594837.710695   36934 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1754594837.726305   36934 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1754594837.726333   36934 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1754594837.726337   36934 computation_placer.cc:177] computation placer alr

Results are saved in runs/train/exp folder

In [ ]:
!python val_dual.py --data /content/drive/MyDrive/Final_Defence/data/dataset.yaml --img 640 --batch 8 --conf 0.001 --iou 0.7 --device 0 --weights /content/drive/MyDrive/Final_Defence/yolov9/runs/train/exp4/weights/best.pt --save-json --name yolov9_c_640_val

val_dual: data=/content/drive/MyDrive/Final_Defence/data/dataset.yaml, weights=['/content/drive/MyDrive/Final_Defence/yolov9/runs/train/exp4/weights/best.pt'], batch_size=8, imgsz=640, conf_thres=0.001, iou_thres=0.7, max_det=300, task=val, device=0, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=True, project=runs/val, name=yolov9_c_640_val, exist_ok=False, half=False, dnn=False, min_items=0
YOLO 🚀 v0.1-104-g5b1ea9a Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
yolov9_custom summary: 580 layers, 60500530 parameters, 0 gradients, 263.9 GFLOPs
val: Scanning /content/drive/MyDrive/Final_Defence/data/labels/val.cache... 273 images, 0 backgrounds, 0 corrupt: 100% 273/273 [00:00<?, ?it/s]
                 Class     Images  Instances          P          R      mAP50   mAP50-95:   3% 1/35 [00:02<01:23,  2.46s/it]Exception in thread Thread-3 (plot_images):
Traceback (most recent call las

##  Inference with Custom Model

In [ ]:
!python detect.py --img 1280 --conf 0.1 --device 0 --weights /content/drive/MyDrive/yolo/BrainTumor/yolov9/runs/train/exp4/weights/best.pt --source /content/drive/MyDrive/yolo/BrainTumor/data/test/images

detect: weights=['/content/drive/MyDrive/yolo/BrainTumor/yolov9/runs/train/exp2/weights/best.pt'], source=/content/drive/MyDrive/yolo/BrainTumor/data/test/images, data=data/coco128.yaml, imgsz=[1280, 1280], conf_thres=0.1, iou_thres=0.45, max_det=1000, device=0, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLO 🚀 v0.1-61-g3e4f970 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
yolov9_custom summary: 580 layers, 60500530 parameters, 0 gradients, 263.9 GFLOPs
Traceback (most recent call last):
  File "/content/drive/.shortcut-targets-by-id/1DIVmoKUCtbzs7mQj-yRzov2Dv-mmXC_z/BrainTumor/yolov9/detect.py", line 231, in <module>
    main(opt)
  File "/content/drive/.shortcut-targets-by-id/1DIVmoKUCtbzs7mQj-yRzov2D

In [34]:
from ultralytics import YOLO
import os

# -----------------------------
# Paths
# -----------------------------
MODEL_PATH = '/content/drive/MyDrive/Final_Defence/yolov9/runs/train/exp4/weights/best.pt'
DATA_YAML  = '/content/drive/MyDrive/Final_Defence/data/dataset.yaml'
RESULT_DIR = '/content/drive/MyDrive/Final_Defence/yolov9'

os.makedirs(RESULT_DIR, exist_ok=True)

# -----------------------------
# Load trained YOLO model
# -----------------------------
model = YOLO(MODEL_PATH)

# -----------------------------
# Test set evaluation with custom save path
# -----------------------------
results = model.val(
    data=DATA_YAML,
    split='test',
    save=True,        # save prediction images
    save_txt=True,    # save predicted bounding boxes
    save_conf=True,   # save confidence scores
    project=RESULT_DIR,  # custom folder
    name='test_results'  # subfolder inside RESULT_DIR
)

Ultralytics 8.3.191 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)


TypeError: BaseModel.fuse() got an unexpected keyword argument 'verbose'

In [ ]:
from ultralytics import YOLO

MODEL_PATH = '/content/drive/MyDrive/Final_Defence/yolov11/train_results3/weights/best.pt'
DATA_YAML  = '/content/drive/MyDrive/Final_Defence/data/dataset.yaml'

model = YOLO(MODEL_PATH)

# Minimal val call
results = model.val(data=DATA_YAML, split='test', save=True)


Ultralytics 8.3.191 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
YOLO11m summary (fused): 125 layers, 20,032,345 parameters, 0 gradients, 67.7 GFLOPs
val: Fast image access ✅ (ping: 1.3±1.2 ms, read: 13.9±29.6 MB/s, size: 697.3 KB)
val: Scanning /content/drive/MyDrive/Final_Defence/data/labels/test.cache... 136 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 136/136 48700.2it/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 9/9 0.69it/s 13.1s
                   all        136        288       0.96      0.946       0.96      0.786
               Blastic         44         63      0.994          1      0.995      0.827
                 Lytic         51        184      0.899      0.837       0.89      0.597
                Normal         41         41      0.988          1      0.995      0.934
Speed: 2.0ms preprocess, 24.1ms inference, 0.0ms loss, 4.1ms postprocess per image
Results saved to runs/dete

In [36]:
results = model.predict(
    source='/content/drive/MyDrive/Final_Defence/data/images/test',
    save=True,        # prediction images save করবে
    save_txt=True,    # bounding boxes txt save হবে
    save_conf=True    # confidence scores save হবে
)



Ultralytics 8.3.191 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)


Fusing layers... 
yolov9_custom summary: 580 layers, 60500530 parameters, 0 gradients, 263.9 GFLOPs


TypeError: DetectionModel.forward() got an unexpected keyword argument 'embed'

In [37]:
from ultralytics import YOLO
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

# -----------------------------
# Paths
# -----------------------------
MODEL_PATH = '/content/drive/MyDrive/Final_Defence/yolov9/runs/train/exp4/weights/best.pt'
TEST_IMAGE_PATH = '/content/drive/MyDrive/Final_Defence/data/images/test'
RESULT_DIR = '/content/drive/MyDrive/Final_Defence/yolov9'

os.makedirs(RESULT_DIR, exist_ok=True)

# -----------------------------
# Load trained YOLO model
# -----------------------------
model = YOLO(MODEL_PATH)

# -----------------------------
# Predict on test set
# -----------------------------
results = model.predict(
    source=TEST_IMAGE_PATH,
    save=True,        # save images with bounding boxes
    save_txt=True,    # save bounding boxes txt
    save_conf=True
)

# -----------------------------
# Confusion Matrix Generation
# -----------------------------
# For demonstration, assume we have true labels and predicted labels as lists
# Replace this with your actual labels extraction from txt or dataset
y_true = []  # List of true labels for all test images
y_pred = []  # List of predicted labels (YOLO outputs)

# Example: if your classes are ['Normal', 'Lytic', 'Blastic']
class_names = ['Normal', 'Lytic', 'Blastic']

# Calculate confusion matrix
cm = confusion_matrix(y_true, y_pred, labels=[0,1,2])

# Plot heatmap
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")

# Save confusion matrix figure
cm_path = os.path.join(RESULT_DIR, "confusion_matrix.png")
plt.savefig(cm_path)
plt.close()

print(f"Predictions saved in {RESULT_DIR}")
print(f"Confusion matrix saved at {cm_path}")


TypeError: BaseModel.fuse() got an unexpected keyword argument 'verbose'